## Skeleton Code

The code below provides a skeleton for the model building & training component of your project. You can add/remove/build on code however you see fit, this is meant as a starting point.

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
import sklearn.model_selection
from random import sample 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, plot_precision_recall_curve, f1_score, confusion_matrix
from tensorflow import keras

## Do some early processing of your metadata for easier model training:

In [ ]:
## Below is some helper code to read all of your full image filepaths into a dataframe for easier manipulation

all_xray_df = pd.read_csv('/data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('/data','images*', '*', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df.sample(3)

In [ ]:
#set unrealistic ages to NaN
all_xray_df.replace(all_xray_df[all_xray_df['Patient Age']>100]['Patient Age'].values,np.nan, inplace = True)

In [ ]:
## Here you may want to create some extra columns in your table with binary indicators of certain diseases 
## rather than working directly with the 'Finding Labels' column

# Todo
def split_labels(df):
    labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for i in labels:
        df[i] = df['Finding Labels'].map(lambda y: 1.0 if i in y else 0)
        
split_labels(all_xray_df)

In [ ]:
#Drop the 'Unnamed: 11' column from the dataset
all_xray_df.drop('Unnamed: 11', axis = 1, inplace = True)

In [ ]:
all_xray_df.head()

In [ ]:
all_xray_df['Pneumonia Class'] = all_xray_df['Pneumonia'].map(lambda x: 'Positive' if x == 1 else 'Negative')


## Create your training and testing data:

In [ ]:
def create_splits(vargs):
    
#Initial split
    train_data, val_data = sklearn.model_selection.train_test_split(vargs, test_size = 0.2, stratify = vargs['Pneumonia'])

#want equal number of +ve/-ve pneumonia cases in train set
    train_p_inds = train_data[train_data.Pneumonia==1].index.tolist()
    train_np_inds = train_data[train_data.Pneumonia==0].index.tolist()

    train_np_sample = sample(train_np_inds,len(train_p_inds))
    train_data = train_data.loc[train_p_inds + train_np_sample]

#want % of +ve pneumonia cases in validation set to be equal to natural occurence of the disease in the main dataset
    val_p_inds = val_data[val_data.Pneumonia==1].index.tolist()
    val_np_inds = val_data[val_data.Pneumonia==0].index.tolist()

# The following code pulls a random sample of non-pneumonia data that's 4 times as big as the pneumonia sample
    val_np_sample = sample(val_np_inds, 4*len(val_p_inds))
    val_data = val_data.loc[val_p_inds + val_np_sample]
    return  train_data, val_data

In [ ]:
#creating the training and validation sets
train_data, val_data = create_splits(all_xray_df)

In [ ]:
len(train_data)

In [ ]:
len(val_data)

In [ ]:
#ANALYSE DISTRIBUTIONS IN TRAINING AND VALIDATION DATASET

In [ ]:
#defining age distribution fnc
def age(df):
    plt.hist(df['Patient Age'], bins = 10,)
    plt.xlabel('age')
    plt.ylabel('Number of People')
    plt.title('Age Distribution in Dataset')

In [ ]:
#age distribution in training set
age(train_data)

In [ ]:
#majority of data is between 10 and 70 years of age

In [ ]:
#age distribution in validation set
age(val_data)

In [ ]:
#function to plot gender demographics
def gender(df):
    df['Patient Gender'].value_counts().plot(kind='bar')
    plt.xlabel('Gender')
    plt.ylabel('Number of People')
    plt.title('Gender Distribution in Dataset')
    
    return df['Patient Gender'].value_counts()

In [ ]:
#invoking gender distribution fnc for train set
train_gender_distribution = gender(train_data)
train_gender_distribution

In [ ]:
#invoking gender distribution fnc for validation set
val_gender_distribution = gender(val_data)
val_gender_distribution

In [ ]:
# def fnc to show position distribution
def image_pos(df):
    df['View Position'].value_counts().plot(kind='bar')
    plt.xlabel('Image Position')
    plt.ylabel('Number of People')
    plt.title('Image Position Distribution')
    return df['View Position'].value_counts()

In [ ]:
#invoking position distribution fnc for training set
train_image_position_distribution = image_pos(train_data)
train_image_position_distribution

In [ ]:
#invoking position distribution fnc for validation set
val_image_position_distribution = image_pos(val_data)
val_image_position_distribution

In [ ]:
#determine the distribution of diseases comorbid with Pneumonia. Plotting the top 30 combinations
train_data[train_data.Pneumonia == 1]['Finding Labels'].value_counts()[0:30].plot(kind = 'bar')
plt.xlabel('Diseases Comorbid with Pneumonia')
plt.ylabel('Number of People')
plt.title('Distribution of Diseases comorbid with Pneumonia')
train_disease_conjunction_pneumonia = train_data[train_data.Pneumonia == 1]['Finding Labels'].value_counts()

In [ ]:
#determine the distribution of diseases comorbid with Pneumonia. Plotting the top 30 combinations
val_data[val_data.Pneumonia == 1]['Finding Labels'].value_counts()[0:30].plot(kind = 'bar')
plt.xlabel('Diseases Comorbid with Pneumonia')
plt.ylabel('Number of People')
plt.title('Distribution of Diseases comorbid with Pneumonia')
val_disease_conjunction_pneumonia = val_data[val_data.Pneumonia == 1]['Finding Labels'].value_counts()

In [ ]:
#Dropping the Pneumonia column since we have the Pneumonia Class column
train_data.drop('Pneumonia', axis = 1, inplace=True)

In [ ]:
#Dropping the Pneumonia column since we have the Pneumonia Class column
val_data.drop('Pneumonia', axis = 1, inplace=True)

In [ ]:
val_data.shape

In [ ]:
train_data.shape

# Now we can begin our model-building & training

#### First suggestion: perform some image augmentation on your data

In [ ]:
def my_image_augmentation():
    
    my_idg = ImageDataGenerator(rescale = 1. / 255.0, horizontal_flip = True, vertical_flip = False, height_shift_range = 0.1, width_shift_range = 0.1, rotation_range = 20,shear_range = 0.1, zoom_range = 0.1)
    
    
    return my_idg

#function to normalize images in validation dataset
def my_image_val_normalize():
    
    my_idg = ImageDataGenerator(rescale = 1. / 255.0, horizontal_flip = False, vertical_flip = False)
    
    
    return my_idg


def make_train_gen(vargs):
    
    ## Create the actual generators using the output of my_image_augmentation for your training data
    ## This generator uses a batch size of 32
    idg = my_image_augmentation()
    train_gen = idg.flow_from_dataframe(dataframe=vargs, directory=None, x_col = 'path',y_col = 'Pneumonia Class' ,class_mode = 'binary',target_size = (224,224), batch_size = 32)
    

    return train_gen

def make_train_gen_2(vargs):
    
    ## Create the actual generators using the output of my_image_augmentation for your training data
    ## This generator uses a batch size of 16
    idg = my_image_augmentation()
    train_gen = idg.flow_from_dataframe(dataframe=vargs, directory=None, x_col = 'path',y_col = 'Pneumonia Class' ,class_mode = 'binary',target_size = (224,224), batch_size = 16)
    

    return train_gen

def make_train_gen_3(vargs):
    
    ## Create the actual generators using the output of my_image_augmentation for your training data
    ## This generator uses a batch size of 64
    idg = my_image_augmentation()
    train_gen = idg.flow_from_dataframe(dataframe=vargs, directory=None, x_col = 'path',y_col = 'Pneumonia Class' ,class_mode = 'binary',target_size = (224,224), batch_size = 64)
    

    return train_gen

# Generator for the validation dataset
def make_val_gen(vargs):
    
    idg = my_image_val_normalize()
    val_gen = idg.flow_from_dataframe(dataframe=vargs, directory=None, x_col = 'path',y_col = 'Pneumonia Class',class_mode = 'binary',target_size = (224,224), batch_size = 256)
    
    return val_gen

In [ ]:
# Create the augmented training dataset with a batch size of 32
train_gen = make_train_gen(train_data)

In [ ]:
# Create the augmented training dataset with a batch size of 16

train_gen_2 = make_train_gen_2(train_data)

In [ ]:
# Create the augmented training dataset with a batch size of 64

train_gen_3 = make_train_gen_3(train_data)

In [ ]:
#Create normalized validation dataset
val_gen = make_val_gen(val_data)

In [ ]:
## May want to pull a single large batch of random validation data for testing after each epoch:
valX, valY = val_gen.next()

In [ ]:
## May want to look at some examples of our augmented training data. 
## This is helpful for understanding the extent to which data is being manipulated prior to training, 
## and can be compared with how the raw data look prior to augmentation

t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    if c_y == 1: 
        c_ax.set_title('Pneumonia')
    else:
        c_ax.set_title('No Pneumonia')
    c_ax.axis('off')

## Build your model: 

Recommendation here to use a pre-trained network downloaded from Keras for fine-tuning

In [ ]:
#Loads the VGG 16 model and freezes all but the last CNN layer, returns the new model
def load_pretrained_model_1():
    
    model = VGG16(include_top=True, weights='imagenet')
    transfer_layer = model.get_layer('block5_pool')
    vgg_model = Model(inputs = model.input, outputs = transfer_layer.output)
    for layer in vgg_model.layers[0:17]:
        layer.trainable = False
    
    
    
    return vgg_model


In [ ]:
##Loads the VGG 16 model and freezes all but the last 2 CNN layer, returns the new model
def load_pretrained_model_2():
    
    model = VGG16(include_top=True, weights='imagenet')
    transfer_layer = model.get_layer('block5_pool')
    vgg_model = Model(inputs = model.input, outputs = transfer_layer.output)
    for layer in vgg_model.layers[0:16]:
        layer.trainable = False
    
    
    
    return vgg_model
    

In [ ]:
#This model has 5 Dense layers and uses a dropout of 0.3 and is added on top of the VGG 16 model with all but its last CNN layer frozen

def build_my_model_1(vgg_model):
    

    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.3))
    my_model.add(Dense(4096, activation = 'relu'))
    my_model.add(Dropout(0.3))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.3))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.3))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

        
    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    
    
    
    return my_model

In [ ]:
#This model has 5 Dense layers and uses a dropout of 0.5 and is added on top of the VGG 16 model with all but its last CNN layer frozen


def build_my_model_2(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(4096, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    
    
        
    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    
 
    
    return my_model

In [ ]:
#This model has 4 Dense layers and uses a dropout of 0.5 and is added on top of the VGG 16 model with all but its last CNN layer frozen


def build_my_model_3(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    
  
    
    return my_model

In [ ]:
##This model has 4 Dense layers and uses a dropout of 0.5 and is added on top of the VGG 16 model with all but its last 2 layers frozen

def build_my_model_4(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

        
    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    
 
    
    return my_model

In [ ]:
#This model uses a lr of 0.001
def build_my_model_5(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

        
    
    optimizer = Adam(lr = .001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    

    
    return my_model


In [ ]:
#This model uses a lr of 0.0005

def build_my_model_6(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

        
    
    optimizer = Adam(lr = .0005)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    

    
    return my_model

In [ ]:
#This model consists of just a single Dense layer
def build_my_model_7(vgg_model):
    
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    
    my_model.add(Dense(1, activation = 'sigmoid'))

        
    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    

    
    return my_model

In [ ]:
#This model will eventually be trained on a training set with batch size = 16
def build_my_model_8(vgg_model):
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    
    # my_model = Sequential()
    # ....add your pre-trained model, and then whatever additional layers you think you might
    # want for fine-tuning (Flatteen, Dense, Dropout, etc.)
    
    # if you want to compile your model within this function, consider which layers of your pre-trained model, 
    # you want to freeze before you compile 
    
    # also make sure you set your optimizer, loss function, and metrics to monitor
    
    # Todo
    
    return my_model

In [ ]:
#This model will eventually be trained on a training set with batch size = 64

def build_my_model_9(vgg_model):
    my_model = Sequential()
    my_model.add(vgg_model)
    my_model.add(Flatten())
    my_model.add(Dropout(0.5))
    my_model.add(Dense(1024, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(512, activation = 'relu'))
    my_model.add(Dropout(0.5))
    my_model.add(Dense(256, activation = 'relu'))
    my_model.add(Dense(1, activation = 'sigmoid'))
    

    
    optimizer = Adam(lr = .0001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    my_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    

    
    return my_model

In [ ]:
weight_path_1="my_model_1-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_1 = ModelCheckpoint(weight_path_1, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_1 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_1 = [checkpoint_1, early_1]

In [ ]:
weight_path_2="my_model_2-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_2 = ModelCheckpoint(weight_path_2, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_2 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_2 = [checkpoint_2, early_2]


In [ ]:
weight_path_3="my_model_3-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_3 = ModelCheckpoint(weight_path_3, monitor= 'val_loss', verbose=1, save_best_only=True,mode= 'min', save_weights_only = True)

early_3 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_3 = [checkpoint_3, early_3]

In [ ]:
weight_path_4="my_model_4-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_4 = ModelCheckpoint(weight_path_4, monitor= 'val_loss', verbose=1,save_best_only=True, mode= 'min', save_weights_only = True)

early_4 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_4 = [checkpoint_4, early_4]

In [ ]:
weight_path_5="my_model_5-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_5 = ModelCheckpoint(weight_path_5, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_5 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_5 = [checkpoint_5, early_5]

In [ ]:
weight_path_6="my_model_6-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_6 = ModelCheckpoint(weight_path_6, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_6 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_6 = [checkpoint_6, early_6]

In [ ]:
weight_path_7="my_model_7-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_7 = ModelCheckpoint(weight_path_7, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_7 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_7 = [checkpoint_7, early_7]

In [ ]:

weight_path_8="my_model_8-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_8 = ModelCheckpoint(weight_path_8, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_8 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_8 = [checkpoint_8, early_8]

In [ ]:
weight_path_9="my_model_9-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint_9 = ModelCheckpoint(weight_path_9, monitor= 'val_loss', verbose=1, save_best_only=True, mode= 'min', save_weights_only = True)

early_9 = EarlyStopping(monitor= 'val_loss', mode= 'min', patience=7)

callbacks_list_9 = [checkpoint_9, early_9]

### Start training! 

In [ ]:
## train your model

# Todo

# history = my_model.fit_generator(train_gen, 
#                           validation_data = (valX, valY), 
#                           epochs = , 
#                           callbacks = callbacks_list)

##### After training for some time, look at the performance of your model by plotting some performance statistics:

Note, these figures will come in handy for your FDA documentation later in the project

In [ ]:
## After training, make some predictions to assess your model's overall performance
## Note that detecting pneumonia is hard even for trained expert radiologists, 
## so there is no need to make the model perfect.
my_model.load_weights(weight_path)
pred_Y = new_model.predict(valX, batch_size = 32, verbose = True)

In [ ]:
def plot_auc(t_y, p_y):
    
    ## Hint: can use scikit-learn's built in functions here like roc_curve
    
    # Todo
    
    return

## what other performance statistics do you want to include here besides AUC? 


# def ... 
# Todo

# def ...
# Todo
    
#Also consider plotting the history of your model training:

def plot_history(history):
    
    # Todo
    return

In [ ]:
## plot figures

# Todo

Once you feel you are done training, you'll need to decide the proper classification threshold that optimizes your model's performance for a given metric (e.g. accuracy, F1, precision, etc.  You decide) 

In [ ]:
## Find the threshold that optimize your model's performance,
## and use that threshold to make binary classification. Make sure you take all your metrics into consideration.

# Todo

In [ ]:
## Let's look at some examples of predicted v. true with our best model: 

# Todo

# fig, m_axs = plt.subplots(10, 10, figsize = (16, 16))
# i = 0
# for (c_x, c_y, c_ax) in zip(valX[0:100], testY[0:100], m_axs.flatten()):
#     c_ax.imshow(c_x[:,:,0], cmap = 'bone')
#     if c_y == 1: 
#         if pred_Y[i] > YOUR_THRESHOLD:
#             c_ax.set_title('1, 1')
#         else:
#             c_ax.set_title('1, 0')
#     else:
#         if pred_Y[i] > YOUR_THRESHOLD: 
#             c_ax.set_title('0, 1')
#         else:
#             c_ax.set_title('0, 0')
#     c_ax.axis('off')
#     i=i+1

In [ ]:
## Just save model architecture to a .json:

model_json = my_model.to_json()
with open("my_model.json", "w") as json_file:
    json_file.write(model_json)